#### Import and process rds frile

In [2]:
from pyreadr import read_r
import pandas as pd
import numpy as np

In [3]:
epa_rds = read_r("rintro-chapter14-epa.rds")
for key in epa_rds.keys():
    epa_df = pd.DataFrame(epa_rds[key])

In [4]:
epa_df.head()

,host,timestamp,request,status,bytes,rawhost,datetime,reqtype,pagetype,page
0,host195,[29:23:53:25],GET /Software.html,http200,1497.0,141.243.1.172,1995-08-30 03:53:25,GET,html,/Software.html
1,host1888,[29:23:53:36],GET /Consumer.html,http200,1325.0,query2.lycos.cs.cmu.edu,1995-08-30 03:53:36,GET,html,/Consumer.html
2,host2120,[29:23:53:53],GET /News.html,http200,1014.0,tanuki.twics.com,1995-08-30 03:53:53,GET,html,/News.html
3,host2273,[29:23:54:15],GET /,http200,4889.0,wpbfl2-45.gate.net,1995-08-30 03:54:15,GET,other,/
4,host2273,[29:23:54:16],GET /icons/circle_logo_small.gif,http200,2624.0,wpbfl2-45.gate.net,1995-08-30 03:54:16,GET,gif,/icons/circle_logo_small.gif


In [5]:
# convert string to datetime object
epa_df['datetime'] = pd.to_datetime(epa_df['datetime'])
epa_df['timestamp'] = [x.timestamp() for x in epa_df['datetime']]

In [6]:
epa_df.status.value_counts()

status
http200    36712
http304     5300
http302     4506
http404      611
http501      272
http403      272
http500       69
http400        6
Name: count, dtype: int64

In [7]:
epa_html200 = epa_df[(epa_df["status"] == "http200") & (epa_df['pagetype'] == "html")]

In [8]:
epa_ordered = epa_html200.sort_values(by=['host', 'timestamp'])

In [9]:
epa_ordered["page"] = [x[:-5] for x in epa_ordered['page']]

In [10]:
session_time = 15*60
timestamp = epa_ordered["timestamp"].to_numpy()
timediff = timestamp[1:] - timestamp[:-1]

epa_ordered.reset_index(drop=True, inplace=True)
epa_ordered["timediff"] = pd.concat([pd.Series([pd.NA]),pd.Series(timediff)], ignore_index=True)

In [11]:
top20_page = epa_ordered["page"].value_counts()[:20].keys()

epa_ordered20 = epa_ordered[ [x in top20_page for x in epa_ordered['page']] ]
epa_ordered20.reset_index(drop=True, inplace=True)

In [12]:
epa_stream = [epa_ordered20.iloc[0, :]["page"]]

session = [1]
for i in range(1,epa_ordered20.shape[0]):
    prev_row = epa_ordered20.iloc[i-1,:]
    curr_row = epa_ordered20.iloc[i,:]
    if prev_row["host"] == curr_row["host"]:
        if curr_row["timediff"] >= session_time:
            epa_stream[-1] += " END"
            session.append(session[-1] + 1)
            epa_stream.append(curr_row['page'])
        else:
            epa_stream[-1] += f" {curr_row['page']}"
            session.append(session[-1])
    else:
        epa_stream[-1] += " END"
        session.append(session[-1] + 1)
        epa_stream.append(curr_row['page'])

epa_ordered20['session'] = session

/tmp/ipykernel_2993/1837779690.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_ordered20['session'] = session


In [14]:
len(epa_stream) == max(epa_ordered20['session'])

True

In [23]:
epa_stream_1 = [x if len(x.split(" ")) > 2 else pd.NA for x in epa_stream]

In [25]:
epa_stream_1 = pd.Series(epa_stream_1).dropna()
epa_stream_1

3                    /docs/WhatsNew /Offices /Offices END
4                              /PIC /docs/titlesearch END
5                              /PIC /docs/titlesearch END
7                                     /Info /Research END
15            /Rules /PIC /information /docs/Internet END
                              ...                        
1207    /docs/major /enviro/html/ef_home /enviro/html/...
1208                 /docs/major /enviro/html/ef_home END
1212              /enviro/html/ef_home /docs/WhatsNew END
1216              /information /docs/Welcome/EPA /PIC END
1220                              /Rules /Initiatives END
Length: 505, dtype: object

In [26]:
markov_chain_data = ""
for session_sequence in epa_stream_1:
    markov_chain_data += f" {session_sequence}" 

In [15]:
epa_ordered.head(10)

,host,timestamp,request,status,bytes,rawhost,datetime,reqtype,pagetype,page,timediff
0,host10,809795255.0,GET /docs/ozone/index.html,http200,3717.0,128.144.102.56,1995-08-30 15:07:35,GET,html,/docs/ozone/index,<NA>
1,host10,809796741.0,GET /Research.html,http200,2500.0,128.144.102.56,1995-08-30 15:32:21,GET,html,/Research,1486.0
2,host10,809796788.0,GET /docs/campus/campus.html,http200,4964.0,128.144.102.56,1995-08-30 15:33:08,GET,html,/docs/campus/campus,47.0
3,host10,809797100.0,GET /docs/BioTech.html,http200,331.0,128.144.102.56,1995-08-30 15:38:20,GET,html,/docs/BioTech,312.0
4,host100,809795012.0,GET /docs/OPPTS_Harmonized/abguide.txt.html,http200,3545.0,131.92.168.211,1995-08-30 15:03:32,GET,html,/docs/OPPTS_Harmonized/abguide.txt,-2088.0
5,host100,809795217.0,GET /OWOW/sec4/index.html,http200,2044.0,131.92.168.211,1995-08-30 15:06:57,GET,html,/OWOW/sec4/index,205.0
6,host100,809795228.0,GET /Rules.html,http200,3273.0,131.92.168.211,1995-08-30 15:07:08,GET,html,/Rules,11.0
7,host100,809795280.0,GET /OWOW/sec301tech/index.html,http200,8866.0,131.92.168.211,1995-08-30 15:08:00,GET,html,/OWOW/sec301tech/index,52.0
8,host100,809795341.0,GET /OWOW/sec301tech/3f.html,http200,22843.0,131.92.168.211,1995-08-30 15:09:01,GET,html,/OWOW/sec301tech/3f,61.0
9,host100,809796477.0,GET /docs/Environment.html,http200,2760.0,131.92.168.211,1995-08-30 15:27:57,GET,html,/docs/Environment,1136.0


#### Markov Chains model

In [29]:
# use markovify
import markovify

model = markovify.Text(epa_stream_1, state_size=1)
for _ in range(5):
    print(model.make_sentence())

/PIC /docs/titlesearch /docs/titlesearch /Research END
/docs/Welcome/EPA /Offices /Rules END
/docs/Internet /docs/Internet /docs/Internet /docs/Environment END
/PIC /docs/WhatsNew /docs/WhatsHot /docs/WhatsHot END
/Standards /Software /enviro/html/ef_home /docs/WhatsHot /docs/WhatsNew /Rules /Initiatives END


In [47]:
model.make_sentence_with_start("/Rules", strict=False)

'/Rules /Offices END'